In [2]:
from dotenv import load_dotenv
import os
#from influxdb_client import InfluxDBClient, Point, WriteOptions
from influxdb import InfluxDBClient as InfluxDBClientV1
from influxdb_client import InfluxDBClient as InfluxDBClientV2
from influxdb_client.client.write_api import SYNCHRONOUS, WriteOptions
from datetime import datetime
#from influxdb_client import InfluxDBClient
# from influxdb_client.client.write_api import SYNCHRONOUS



# https://thenewstack.io/getting-started-with-python-and-influxdb/
# from influxdb import InfluxDBClient

# influx v1.8 - old/current location

load_dotenv("local.env")

user = os.getenv("INFLUX_USER")
password = os.getenv("INFLUX_PASS")
dbname = os.getenv("INFLUX_DBNAME")
host1 = "iotdash.austinvoicedata.com"

client1 = InfluxDBClientV1(
    host=host1,
    path="influxdb:8086",
    port=443,
    ssl=True,
    verify_ssl=True,
    username=user,
    password=password,
    database=dbname
)
meas = "xirka"

In [10]:
"""
Configure credentials for Influx Cloud
"""
influx_cloud_url = 'eastus-1.azure.cloud2.influxdata.com'
influx_cloud_token = os.getenv("INFLUXCLOUD_TOKEN")
bucket = 'bucket2'
org = 'Everynet-IoTDashboard'

# client2 = influxdb_client.InfluxDBClient(url=influx_cloud_url, token=influx_cloud_token)

client2 = InfluxDBClientV2(
    url=F'https://{influx_cloud_url}',
    token=influx_cloud_token,
    org=org,
    # debug=True
)

# write_api = client2.write_api(write_options=SYNCHRONOUS)
write_api = client2.write_api(write_options=WriteOptions(batch_size=500,
                                                      flush_interval=10_000,
                                                      jitter_interval=2_000,
                                                      retry_interval=5_000,
                                                      max_retries=5,
                                                      max_retry_delay=30_000,
                                                      exponential_base=2))
#p = Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)
# write_api.write(bucket=bucket, record=p)
# delete bucket contents for 2022 from CLI
# influx delete -b radiobridge0 --start 2022-01-01T11:00:00Z --stop 2023-01-01T00:00:00Z
# query the bucket
# influx query 'from(bucket:"radiobridge0") |> range(start:-1mo)'

In [11]:


query_where = 'select * from radiobridge where dev_eui=$deveui'
bind_params = {'deveui': '0080000004017e61'}
query = F'SELECT * FROM {meas}'

result = client1.query(query,epoch="ns")
# result = client1.query(query_where, bind_params=bind_params)

# print("Result:\n{0}".format(result))
points = list(result.get_points())
print(F'Got {len(points)} Data Points')
# print("Points:\n{0}".format(points))

Got 37705 Data Points


In [12]:
# print("Points:\n{0}".format(points))

data = []

i = 0
for p in points:
    i += 1
    #print(p)

    linehead = F'{meas},dev_eui={p["dev_eui"]},gateway_eui={p["gateway_eui"]}'
    metrics = 'xfer=True'
    
    if 'rssi' in p and p['rssi'] is not None:
        metrics = metrics + F',rssi={int(p["rssi"])}i'
    if 'snr' in p and p['snr'] is not None:
        metrics = metrics + F',snr={round(p["snr"],1)}'
    if 'bandwidth' in p and p['bandwidth'] is not None:
        metrics = metrics + F',bandwidth={int(p["bandwidth"])}i'
    if 'frequency' in p and p['frequency'] is not None:
        metrics = metrics + F',frequency={round(p["frequency"],1)}'
    if 'spreading_factor' in p and p['spreading_factor'] is not None:
        metrics = metrics + F',spreading_factor={int(p["spreading_factor"])}i'
    if 'rcv_time' in p and p['rcv_time'] is not None:
        metrics = metrics + F',rcv_time={p["rcv_time"]}'   
    if 'f_count' in p and p['f_count'] is not None:
        metrics = metrics + F',f_count={int(p["f_count"])}i'
        
    if 'frame_size' in p and p['frame_size'] is not None:
        metrics = metrics + F',frame_size={int(p["frame_size"])}i'
    if 'battery_voltage' in p and p['battery_voltage'] is not None:
        metrics = metrics + F',battery_voltage={p["battery_voltage"]}'
    if 'gw_alt' in p and p['gw_alt'] is not None:
        metrics = metrics + F',gw_alt={int(p["gw_alt"])}i'
    if 'gw_latitude' in p and p['gw_latitude'] is not None:
        metrics = metrics + F',gw_latitude={round(p["gw_latitude"],5)}'
    if 'gw_longitude' in p and p['gw_longitude'] is not None:
        metrics = metrics + F',gw_longitude={round(p["gw_longitude"],5)}'
    if 'temperature' in p and p['temperature'] is not None:
        metrics = metrics + F',temperature={round(p["temperature"],1)}'
    if 'humidity' in p and p['humidity'] is not None:
        metrics = metrics + F',humidity={round(p["humidity"],1)}'
    if 'message_type' in p and p['message_type'] is not None:
        metrics = metrics + F',message_type=\"{p["message_type"]}\"'

    before=datetime.now()
    record = linehead + " " + metrics + " " + str(p['time'])
    data.append(record)
    #write_api.write(bucket=bucket, record=record)
    #duration=datetime.now()-before
    #print(F'R{i}:t{round(duration.total_seconds(),3)}s:{record}')
write_api.write(bucket=bucket, record=data)
# print(data)